In [6]:
import os
import time

import openai
from dotenv import load_dotenv
from openai import OpenAI
from PIL import Image

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]


client = OpenAI()

file = client.files.create(
    file=open("world_happiness.csv", "rb"), purpose="assistants"
)

assistant = client.beta.assistants.create(
    name="Data visualizer",
    description="You are excellent at handling missing data. You analyze .csv files for missing values and impute them appropriately.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
)

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Impute any missing data in this file.",
            "attachments": [
                {"file_id": file.id, "tools": [{"type": "code_interpreter"}]}
            ],
        }
    ]
)

run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
print(f"👉 Run Created: {run.id}")

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"🏃🏻 Run Status: {run.status}")
    time.sleep(1)
else:
    print("🏁 Run Completed!")


👉 Run Created: run_dc8ybcIz9C9UnCjGZViy6jyB
🏃🏻 Run Status: queued
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: in_progress
🏃🏻 Run Status: completed
🏁 Run Completed!


In [7]:
message_response = client.beta.threads.messages.list(thread_id=thread.id)



In [12]:
message_response.data[0]

Message(id='msg_3utq2LCYfbRtqnTPy9NWqQCp', assistant_id='asst_Ylx11K9hdOqf9a4wf7bEAwTG', attachments=[Attachment(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=160, file_path=FilePath(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm'), start_index=123, text='sandbox:/mnt/data/imputed_dataset.csv', type='file_path')], value='The imputed dataset has been saved successfully. You can download it using the link below:\n\n[Download the imputed dataset](sandbox:/mnt/data/imputed_dataset.csv)'), type='text')], created_at=1718095207, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_dc8ybcIz9C9UnCjGZViy6jyB', status=None, thread_id='thread_H8EkKm0VqNKzimMAGaA78CB9')

In [13]:
file = client.files.content(
    message_response.data[0]
)

NotFoundError: Error code: 404 - {'error': {'message': "Invalid URL (GET /v1/files/Message(id='msg_3utq2LCYfbRtqnTPy9NWqQCp', assistant_id='asst_Ylx11K9hdOqf9a4wf7bEAwTG', attachments=[Attachment(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=160, file_path=FilePath(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm'), start_index=123, text='sandbox:/mnt/data/imputed_dataset.csv', type='file_path')], value='The imputed dataset has been saved successfully. You can download it using the link below:\\n\\n[Download the imputed dataset](sandbox:/mnt/data/imputed_dataset.csv)'), type='text')], created_at=1718095207, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_dc8ybcIz9C9UnCjGZViy6jyB', status=None, thread_id='thread_H8EkKm0VqNKzimMAGaA78CB9')/content)", 'type': 'invalid_request_error', 'para

In [10]:
file = client.files.content(
    message_response.data[0].content[0]
)
image_data_bytes = file.read()

with open("my-image.png", "wb") as file:
    file.write(image_data_bytes)


image = Image.open("my-image.png")
image.show()


NotFoundError: Error code: 404 - {'error': {'message': "Invalid URL (GET /v1/files/Message(id='msg_3utq2LCYfbRtqnTPy9NWqQCp', assistant_id='asst_Ylx11K9hdOqf9a4wf7bEAwTG', attachments=[Attachment(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=160, file_path=FilePath(file_id='file-bJ1WlTw1t61hKAtsJUP0XrIm'), start_index=123, text='sandbox:/mnt/data/imputed_dataset.csv', type='file_path')], value='The imputed dataset has been saved successfully. You can download it using the link below:\\n\\n[Download the imputed dataset](sandbox:/mnt/data/imputed_dataset.csv)'), type='text')], created_at=1718095207, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_dc8ybcIz9C9UnCjGZViy6jyB', status=None, thread_id='thread_H8EkKm0VqNKzimMAGaA78CB9')/content)", 'type': 'invalid_request_error', 'para

In [ ]:
with open("my-image.png", "wb") as file:
    file.write(image_data_bytes)


image = Image.open("my-image.png")
image.show()
